**IMPORT REQUIRED LIBRARIES**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import pathlib
import glob
import shutil
import os
import random

**CONNECTING TO WANDB**

In [2]:
#---------------------------------------install and import wandb -------------------------------------------------
!pip install wandb -qqq
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 2.1MB 7.9MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 133kB 32.7MB/s 
     |████████████████████████████████| 163kB 34.0MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 


In [3]:
#--------------------------------------------login to wandb --------------------------------------------
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
#--------------------------------caution: terminal commands ---------------------------------------------
%cd
%cd .keras/datasets/
!rm -r *

/root
[Errno 2] No such file or directory: '.keras/datasets/'
/root
rm: cannot remove '*': No such file or directory


In [5]:
########################################### download data from given url ###############################################

dataset_url = "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"
data_dir = tf.keras.utils.get_file('dakshina_dataset_v1.0', origin=dataset_url, untar=True)

2008342528/2008340480 [==============================] - 17s 0us/step


In [6]:
#----------------------------------terminal command -----------------------------------------------
%cd /root/.keras/datasets/dakshina_dataset_v1.0/hi/lexicons/
!ls

/root/.keras/datasets/dakshina_dataset_v1.0/hi/lexicons
hi.translit.sampled.dev.tsv   hi.translit.sampled.train.tsv
hi.translit.sampled.test.tsv


In [7]:
train_data_path = "hi.translit.sampled.train.tsv"
test_data_path = "hi.translit.sampled.test.tsv"
validation_data_path = "hi.translit.sampled.dev.tsv"

**UTILITY FUNCTION FOR DATA PRE-PROCESSING**

In [8]:
################################################ preparing the data in required format #################################################

def data(path,input_token_index,target_token_index):
  input_texts = []
  target_texts = []
  with open(path, "r", encoding="utf-8") as f:
      lines = f.read().split("\n")
  del lines[-1]
  for line in lines:
      target_text, input_text, _ = line.split("\t")

      input_text = "\t"+input_text + "\n"
      target_text = "\t"+target_text + "\n"

      input_texts.append(input_text)
      target_texts.append(target_text)

  encoder_input_data = np.zeros( (len(input_texts), max_encoder_seq_length), dtype="int32")
  decoder_input_data = np.zeros( (len(input_texts), max_decoder_seq_length), dtype="int32")
  decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="int32")

  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t] = input_token_index[char]
      encoder_input_data[i, t + 1 :] = input_token_index[" "]
      for t, char in enumerate(target_text):
          # decoder_target_data is ahead of decoder_input_data by one timestep
          decoder_input_data[i, t] = target_token_index[char]
          if t > 0:
              # decoder_target_data will be ahead by one timestep
              # and will not include the start character.
              decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :] = target_token_index[" "]
      decoder_target_data[i, t:, target_token_index[" "]] = 1.0
    
  return (encoder_input_data, decoder_input_data, decoder_target_data)

**PREPROCESSING THE DATA**

In [9]:
################################################ preprocessing the train data and getting dictionaries #################################################

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(train_data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

del lines[-1]
for line in lines:
    target_text, input_text, _ = line.split("\t")
    input_text = "\t"+input_text + "\n"
    target_text = "\t"+target_text + "\n"

    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters.add(" ")
target_characters.add(" ")

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])


Number of samples: 44204
Number of unique input tokens: 29
Number of unique output tokens: 66
Max sequence length for inputs: 22
Max sequence length for outputs: 21



**MACHINE TRANSLITERATOR**


In [10]:
class Machine_Transliterator():

  ############################################# constructor for class Machine_Transliterator ##########################################

  def __init__(self,max_encoder_seq_length,max_decoder_seq_length,encoder_embed_size, decoder_embed_size,
               num_encoder_layers,num_decoder_layers,epochs, hidden_layer_size,
               num_encoder_tokens, cell_type, num_decoder_tokens,input_token_index, target_token_index, 
               activation="softmax",optimizer="rmsprop",dropout=0.05):
    
    self.cell_type= cell_type
    self.hidden_layer_size = hidden_layer_size  
    self.optimizer = optimizer
    self.activation = activation   
    self.dropout=dropout

    #-------------------------------------- Number of hidden layers -------------------------------------

    self.num_encoder_layers = num_encoder_layers
    self.num_decoder_layers=num_decoder_layers

    #-------------------------------------- sequence length -------------------------------------
    self.max_decoder_seq_length=max_decoder_seq_length
    self.max_encoder_seq_length=max_encoder_seq_length

    #---------------------------------------------Embedding size-------------------------------------
    self.encoder_embed_size = encoder_embed_size
    self.decoder_embed_size = decoder_embed_size
    
    #-----------------information obtained after preprocessing of data-------------------------------------
    self.num_encoder_tokens = num_encoder_tokens
    self.num_decoder_tokens = num_decoder_tokens

    #-----------------------------dictionaries----------------------------------------------------
    self.input_token_index = input_token_index
    self.target_token_index = target_token_index

 


#########################################function to build model ###########################################

  def build_model(self):

    
    # encoder_inputs = keras.Input(shape=(None,))  
    encoder_inputs = keras.Input(shape=(self.max_encoder_seq_length,)) 
    # encoder_embedding_output = tf.keras.layers.Embedding(self.num_encoder_tokens, self.encoder_embed_size)(encoder_inputs)
    encoder_embedding_output = tf.keras.layers.Embedding(input_dim = self.num_encoder_tokens, output_dim = self.encoder_embed_size)(encoder_inputs)


#------------------------------ if cell type = LSTM -------------------------------------------------------------
    if self.cell_type == "lstm":
     #--------------------- encoder -----------------------------------
      encoder = keras.layers.LSTM(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
      encoder_outputs, state_h, state_c = encoder(encoder_embedding_output)
      for i in range(1,self.num_encoder_layers):
        encoder = keras.layers.LSTM(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
        encoder_outputs, state_h, state_c = encoder(encoder_outputs)
      encoder_states = [state_h, state_c]

      #---------------------------decoder ---------------------------------------------------
      decoder_inputs = keras.Input(shape=(self.max_decoder_seq_length, ))      
      # decoder_embedding_output = tf.keras.layers.Embedding(self.num_decoder_tokens, self.decoder_embed_size)(decoder_inputs)
      decoder_embedding_output = tf.keras.layers.Embedding(input_dim = self.num_decoder_tokens, output_dim = self.decoder_embed_size)(decoder_inputs)

      decoder = keras.layers.LSTM(self.hidden_layer_size, return_sequences=True, return_state=True,dropout=self.dropout,use_bias=True)
      decoder_outputs, _, _= decoder(decoder_embedding_output, initial_state = encoder_states)
      for i in range(1,self.num_decoder_layers):
        decoder = keras.layers.LSTM(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
        decoder_outputs, _ , _= decoder(decoder_outputs, initial_state = encoder_states)



#------------------------------ if cell type = Simple RNN -------------------------------------------------------------
    elif self.cell_type == "rnn":
      #--------------------- encoder -----------------------------------
      encoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout, use_bias=True)
      encoder_outputs, state = encoder(encoder_embedding_output)
      for i in range(1,self.num_encoder_layers):
        encoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout, use_bias=True)
        encoder_outputs, state = encoder(encoder_outputs)
      encoder_states = [state]

      #---------------------------decoder ---------------------------------------------------
      decoder_inputs = keras.Input(shape=(self.max_decoder_seq_length,))      
      # decoder_embedding_output = tf.keras.layers.Embedding(self.num_decoder_tokens, self.decoder_embed_size)(decoder_inputs)
      decoder_embedding_output = tf.keras.layers.Embedding(input_dim = self.num_decoder_tokens, output_dim = self.decoder_embed_size)(decoder_inputs)

      decoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True, dropout=self.dropout, use_bias=True)
      decoder_outputs, _ = decoder(decoder_embedding_output, initial_state = encoder_states)
      for i in range(1,self.num_decoder_layers):
        decoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True, dropout=self.dropout, use_bias=True)
        decoder_outputs, _= decoder(decoder_outputs, initial_state = encoder_states)



#------------------------------ if cell type = GRU -------------------------------------------------------------
    elif self.cell_type == "gru":
      #--------------------- encoder -----------------------------------
      encoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
      encoder_outputs, state = encoder(encoder_embedding_output)
      for i in range(1,self.num_encoder_layers):
        encoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
        encoder_outputs, state = encoder(encoder_outputs)
      encoder_states = [state]

      #---------------------------decoder ---------------------------------------------------
      decoder_inputs = keras.Input(shape=(self.max_decoder_seq_length, ))      
      # decoder_embedding_output = tf.keras.layers.Embedding(self.num_decoder_tokens, self.decoder_embed_size)(decoder_inputs)
      decoder_embedding_output = tf.keras.layers.Embedding(input_dim = self.num_decoder_tokens, output_dim = self.decoder_embed_size)(decoder_inputs)
      
      decoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True, dropout=self.dropout,use_bias=True)
      decoder_outputs, _ = decoder(decoder_embedding_output, initial_state = encoder_states)
      for i in range(1,self.num_decoder_layers):
        decoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
        decoder_outputs, _ = decoder(decoder_outputs, initial_state = encoder_states)


    decoder_dense = keras.layers.Dense(self.num_decoder_tokens, activation = self.activation,use_bias=True)
    decoder_outputs = decoder_dense(decoder_outputs)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    #-----------------compile the model -------------------------------------
    model.compile(
         optimizer=self.optimizer,
         loss="categorical_crossentropy",
         metrics=["accuracy"]
         ) 

#-------------------------- return final model ---------------------------------------------------------
    return model





#########################################function for training the model ###########################################

  def train_model(self,encoder_input_data,decoder_input_data,decoder_target_data,epochs,batch_size,
                  val_encoder_input_data, val_decoder_input_data, val_decoder_target_data):
    
     model=self.build_model()
  
     model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size = batch_size,
        epochs = epochs,
        validation_data = ([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data),
        callbacks=[WandbCallback()]
        )
     return model

#===================================== end of class Machine_Transliterator ==========================================



**PARAMETERS**

In [11]:
cell_type = "rnn" # Type of the recurring unit
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
hidden_layer_size= 128  # Latent dimensionality of the encoding space.
# num_samples = 10000  # Number of samples to train on.
activation = "softmax" #activation
optimizer = "Adam"  #optimizer
encoder_embed_size = 32 #Encoder embedsize
decoder_embed_size = 32 #Decoder embedsize
num_encoder_layers=3  # number of hidden layers in encoder
num_decoder_layers=3   # number of hidden layers in decoder
dropout=0.01#Dropout

**CREATING MACHINE TRANSLITERATOR**

In [12]:
# ########################### creating machine transliterator object ###############################
# machine = Machine_Transliterator(
#     max_encoder_seq_length,max_decoder_seq_length,encoder_embed_size, 
#     decoder_embed_size,num_encoder_layers,num_decoder_layers,
#     batch_size, hidden_layer_size, num_encoder_tokens, cell_type, num_decoder_tokens, 
#      input_token_index,target_token_index, activation, optimizer,dropout
#     )

**PREPROCESSING THE DATA**

In [13]:
(encoder_input_data,decoder_input_data,decoder_target_data)=data(train_data_path,input_token_index,target_token_index)


(val_encoder_input_data,val_decoder_input_data,val_decoder_target_data)=data(validation_data_path ,input_token_index,target_token_index)

In [14]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)
print(val_encoder_input_data.shape)
print(val_decoder_input_data.shape)
print(val_decoder_target_data.shape)

(44204, 22)
(44204, 21)
(44204, 21, 66)
(4358, 22)
(4358, 21)
(4358, 21, 66)


**TRAINING**



In [15]:
# model = machine.train_model(
#     encoder_input_data, decoder_input_data,decoder_target_data,epochs,batch_size,
#     val_encoder_input_data, val_decoder_input_data, val_decoder_target_data
#     )

In [16]:
# tf.keras.utils.plot_model(model,to_file='model.png',show_shapes=True)

In [17]:
cell_type = ["lstm","rnn","gru"]
batch_size = [16,32,64,128]  # Batch size for training.
epochs = [10,20,30,50]  # Number of epochs to train for.
hidden_layer_size= [128,256,512]  # Latent dimensionality of the encoding space.
activation = "softmax"
optimizer = "Adam"
encoder_embed_size = [27,64] #Encoder embedsize
decoder_embed_size = [27,64] #Decoder embedsize
num_encoder_layers=[1,2,3,4]  # number of hidden layers in encoder
num_decoder_layers=[1,2,3,4]    # number of hidden layers in decoder
dropout=[0.00,0.01, 0.5,0.0001] #Dropout

In [18]:
sweep_config={
              "method":"random",
              'metric' : {
                            'name' : 'val_accuracy',
                            'goal' : 'maximize',
                         },
          "parameters" : {
                            "cell_type":{"values":["lstm","rnn", "gru"]},
                            "batch_size":{"values": [32,64,128]},
                            "epochs":{"values":[30,40,60]}, 
                            "hidden_layer_size":{"values": [128,256,512]}, 
                            "encoder_embed_size": {"values": [27,64] },
                            "decoder_embed_size": {"values":[27,64]  },
                            "num_encoder_layers": {"values": [1,2,3,4]},
                            "num_decoder_layers":{"values":[1,2,3,4] },
                            "dropout": {"values":[0.00,0.1, 0.05,0.001]}
                         }
              }
sweep_id = wandb.sweep(sweep_config, project="Transliterator")

Create sweep with ID: kewrjmtp
Sweep URL: https://wandb.ai/anshikag_2210/Transliterator/sweeps/kewrjmtp


In [19]:
def run():

  wb = wandb.init()
  config = wb.config

  #----------------------sweep parameters------------------------------------
  cell_type =config.cell_type 
  batch_size = config.batch_size 
  epochs = config.epochs 
  hidden_layer_size= config.hidden_layer_size
  encoder_embed_size =config.encoder_embed_size
  decoder_embed_size = config.decoder_embed_size
  num_encoder_layers=config.num_encoder_layers
  num_decoder_layers= config.num_decoder_layers 
  dropout=config.dropout
  
  ########################### creating machine transliterator object ###############################
  machine = Machine_Transliterator(
      max_encoder_seq_length,max_decoder_seq_length,encoder_embed_size, 
      decoder_embed_size,num_encoder_layers,num_decoder_layers,
      batch_size, hidden_layer_size, num_encoder_tokens, cell_type, num_decoder_tokens, 
      input_token_index,target_token_index, activation, optimizer,dropout
      )
  model = machine.train_model(
      encoder_input_data, decoder_input_data,decoder_target_data,epochs,batch_size,
      val_encoder_input_data, val_decoder_input_data, val_decoder_target_data
      )  
  return

In [ ]:
wandb.agent(sweep_id, run)

wandb: Agent Starting Run: 4ni70lb6 with config:
wandb: 	batch_size: 32
wandb: 	cell_type: lstm
wandb: 	decoder_embed_size: 64
wandb: 	dropout: 0
wandb: 	encoder_embed_size: 64
wandb: 	epochs: 30
wandb: 	hidden_layer_size: 512
wandb: 	num_decoder_layers: 1
wandb: 	num_encoder_layers: 2
wandb: Currently logged in as: anshikag_2210 (use `wandb login --relogin` to force relogin)


Epoch 1/30
1382/1382 [==============================] - 58s 17ms/step - loss: 1.0573 - accuracy: 0.7339 - val_loss: 0.5763 - val_accuracy: 0.8314
Epoch 2/30
1382/1382 [==============================] - 22s 16ms/step - loss: 0.4817 - accuracy: 0.8560 - val_loss: 0.2546 - val_accuracy: 0.9205
Epoch 3/30
1382/1382 [==============================] - 22s 16ms/step - loss: 0.2088 - accuracy: 0.9353 - val_loss: 0.1876 - val_accuracy: 0.9414
Epoch 4/30
1382/1382 [==============================] - 22s 16ms/step - loss: 0.1328 - accuracy: 0.9584 - val_loss: 0.1693 - val_accuracy: 0.9480
Epoch 5/30
1382/1382 [==============================] - 22s 16ms/step - loss: 0.0946 - accuracy: 0.9702 - val_loss: 0.1616 - val_accuracy: 0.9501
Epoch 6/30
1382/1382 [==============================] - 22s 16ms/step - loss: 0.0714 - accuracy: 0.9773 - val_loss: 0.1641 - val_accuracy: 0.9501
Epoch 7/30
1382/1382 [==============================] - 22s 16ms/step - loss: 0.0547 - accuracy: 0.9829 - val_loss: 0.1685 -

epoch,29
loss,0.01848
accuracy,0.99447
val_loss,0.21927
val_accuracy,0.95268
_runtime,707
_timestamp,1619766510
_step,29
best_val_loss,0.16159
best_epoch,4


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▄▆▇▇▇████████████████████████
val_loss,█▃▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
val_accuracy,▁▆▇███████████████████████████
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███


wandb: Agent Starting Run: nzamark8 with config:
wandb: 	batch_size: 64
wandb: 	cell_type: lstm
wandb: 	decoder_embed_size: 64
wandb: 	dropout: 0.05
wandb: 	encoder_embed_size: 64
wandb: 	epochs: 40
wandb: 	hidden_layer_size: 128
wandb: 	num_decoder_layers: 4
wandb: 	num_encoder_layers: 2


...

Epoch 1/30
1382/1382 [==============================] - 45s 26ms/step - loss: 1.2377 - accuracy: 0.6998 - val_loss: 0.8663 - val_accuracy: 0.7606
Epoch 2/30
1382/1382 [==============================] - 34s 24ms/step - loss: 0.8810 - accuracy: 0.7567 - val_loss: 0.7542 - val_accuracy: 0.7830
Epoch 3/30
1382/1382 [==============================] - 34s 25ms/step - loss: 0.7727 - accuracy: 0.7792 - val_loss: 0.6927 - val_accuracy: 0.8002
Epoch 4/30
1382/1382 [==============================] - 34s 25ms/step - loss: 0.6973 - accuracy: 0.7982 - val_loss: 0.6290 - val_accuracy: 0.8165
Epoch 5/30
1382/1382 [==============================] - 34s 24ms/step - loss: 0.6353 - accuracy: 0.8149 - val_loss: 0.5693 - val_accuracy: 0.8312
Epoch 6/30
1382/1382 [==============================] - 34s 25ms/step - loss: 0.5781 - accuracy: 0.8289 - val_loss: 0.5209 - val_accuracy: 0.8452
Epoch 7/30
1382/1382 [==============================] - 34s 25ms/step - loss: 0.5185 - accuracy: 0.8449 - val_loss: 0.4568 -

epoch,29
loss,0.07924
accuracy,0.974
val_loss,0.20937
val_accuracy,0.94204
_runtime,1040
_timestamp,1619775028
_step,29
best_val_loss,0.20164
best_epoch,22


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▇▆▅▅▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▂▃▃▄▄▄▅▅▆▆▆▇▇▇▇▇▇▇███████████
val_loss,█▇▆▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▃▃▄▄▅▆▆▆▇▇▇▇▇███████████████
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇██
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇██
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███


wandb: Agent Starting Run: qrpjq05e with config:
wandb: 	batch_size: 32
wandb: 	cell_type: rnn
wandb: 	decoder_embed_size: 64
wandb: 	dropout: 0.001
wandb: 	encoder_embed_size: 64
wandb: 	epochs: 60
wandb: 	hidden_layer_size: 128
wandb: 	num_decoder_layers: 4
wandb: 	num_encoder_layers: 3


Epoch 1/60
1382/1382 [==============================] - 204s 145ms/step - loss: 1.0291 - accuracy: 0.7403 - val_loss: 0.5980 - val_accuracy: 0.8317
Epoch 2/60
1382/1382 [==============================] - 199s 144ms/step - loss: 0.5855 - accuracy: 0.8343 - val_loss: 0.4874 - val_accuracy: 0.8579
Epoch 3/60
1382/1382 [==============================] - 197s 143ms/step - loss: 0.4782 - accuracy: 0.8592 - val_loss: 0.4447 - val_accuracy: 0.8677
Epoch 4/60
1382/1382 [==============================] - 196s 142ms/step - loss: 0.4213 - accuracy: 0.8738 - val_loss: 0.3934 - val_accuracy: 0.8836
Epoch 5/60
1382/1382 [==============================] - 195s 141ms/step - loss: 0.3824 - accuracy: 0.8845 - val_loss: 0.3930 - val_accuracy: 0.8841
Epoch 6/60
1382/1382 [==============================] - 197s 143ms/step - loss: 0.3581 - accuracy: 0.8910 - val_loss: 0.3610 - val_accuracy: 0.8924
Epoch 7/60
1382/1382 [==============================] - 201s 145ms/step - loss: 0.3378 - accuracy: 0.8963 - val_